## df.query
https://blog.csdn.net/AlanGuoo/article/details/88874742

df.query(expr, inplace=False, **kwargs)
参数：
- expr: str要评估的查询字符串。 可以引用变量，在前面加@字符
- inplace-False: 是否修改数据或返回副本
- kwargs: dict关键字参数

In [8]:
import pandas as pd
import numpy as np
import numexpr

In [26]:
df=pd.DataFrame(np.arange(12).reshape(3,4), columns=list('ABCD'))
df

,A,B,C,D
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11


In [19]:
## r1 == r2 == r3 == r4 == r5
## 不同点：eval表达式为逻辑，结果返回bool数组，
r1=df[(df.A<8)&(df.B<9)]
r2=pd.eval('df[(df.A<8)&(df.B<9)]')
r3=df[df.eval('A<8 & B<9')]
a=df.A
b=df.B
r5=df[numexpr.evaluate('(a<8) & (b<9)')]


In [21]:
## avg1 == avg2
## @符号标记本地变量
c_avg = df['C'].mean()
avg1= df[(df.A < c_avg) & (df.B < c_avg)]
avg2= df.query('A < @c_avg and B < @c_avg')


,A,B,C,D
0,0,1,2,3
1,4,5,6,7


In [38]:
# 列名
df.query('(A < B) & (B < C)')
# index/ ilevel_0索引 & 列名
df.query('index<B and B<C')
df.query('index > 1')
df.query('ilevel_0>1')

,A,B,C,D
2,8,9,10,11


In [55]:
## 多索引 multiIndex
colors = np.random.choice(['red','blue'], size=6)
foods = np.random.choice(['eggs','meat'], size=6)
Index = pd.MultiIndex.from_arrays([colors,foods], names=['color','food'])
df = pd.DataFrame(np.arange(12).reshape(6,2),index=Index)
print(df)
# 索引名
#df.query('color == "red"')
df.index.names = [None,None]
# ilevel_0 第0级的索引级别
rd = df.query('ilevel_0 == "red"')
print('rd:', '\n', rd)
# ilevel_1 第1级的索引级别
mt = df.query('ilevel_1 == "meat"')
print('mt:', '\n', mt)

             0   1
color food        
red   meat   0   1
blue  meat   2   3
red   eggs   4   5
blue  eggs   6   7
      eggs   8   9
red   meat  10  11
rd: 
            0   1
red meat   0   1
    eggs   4   5
    meat  10  11
mt: 
             0   1
red  meat   0   1
blue meat   2   3
red  meat  10  11


In [61]:
## 多数据df - 具有相同列名（或索引级别、名称)
df1 = pd.DataFrame(np.arange(12).reshape(4,3), columns=list('ABC')) +10
df2 = df1 + 10
print(df1,'\n', df2)
expr = '19<=A<=C<=22'
result = list(map(lambda frame: frame.query(expr),[df1,df2]))
result

    A   B   C
0  10  11  12
1  13  14  15
2  16  17  18
3  19  20  21 
     A   B   C
0  20  21  22
1  23  24  25
2  26  27  28
3  29  30  31


[    A   B   C
 3  19  20  21,
     A   B   C
 0  20  21  22]

In [65]:
## r1 == r2 == r3 == r4 == r5
## 语法类似numpy
df = pd.DataFrame(np.random.randint(10,size=(10,3)),columns = list('ABC'))

r1 = df.query('(A<B) & (B<C)')
r2 = df[(df.A<df.B) & (df.B<df.C)]
r3 = df.query('A<B & B<C')
r4 = df.query('A<B and B<C')
r5 = df.qeury('A<B<C')

,A,B,C
0,0,8,7
1,6,7,3
2,0,1,4
3,3,1,9
4,6,9,3
5,9,1,0
6,3,8,5
7,3,0,0
8,4,5,2
9,4,9,6


In [73]:
## s1 == s2
## '==' 等价于 .isin()
s1 = df.query('B == [8,7,9]')
s2 = df[df.B.isin([8,7,9])]
print(s1,'\n', s2)

## '=='/'!=' vs. 'in'/'not in'
df.query('C == [1,2]')
df.query('[1,2] in C')
df[df.C.isin([1,2])]

df.query('C != [1,2]')
df.query('[1,2] not in C')

   A  B  C
0  0  8  7
1  6  7  3
4  6  9  3
6  3  8  5
9  4  9  6 
    A  B  C
0  0  8  7
1  6  7  3
4  6  9  3
6  3  8  5
9  4  9  6


,A,B,C
0,0,8,7
1,6,7,3
2,0,1,4
3,3,1,9
4,6,9,3
5,9,1,0
6,3,8,5
7,3,0,0
9,4,9,6


In [86]:
## in 与 not in
df = pd.DataFrame({'a': list('abcdef'), 'b': list('fedfed'), 'c':5, 'd':5})
print(df)

## s1 == s2
s1 = df.query('a in b and c == d')
s2 = df[df.a.isin(df.b) & (df.c == df.d)]

r1 = df[df.a.isin(df.b)]
r2 = df.query('a not in b')
r3 = df[~df.a.isin(df.b)]
print('r1:','\n',r1,'\n','r2:','\n',r2,'\n','r3:','\n',r3)

   a  b  c  d
0  a  f  5  5
1  b  e  5  5
2  c  d  5  5
3  d  f  5  5
4  e  e  5  5
5  f  d  5  5
r1: 
    a  b  c  d
3  d  f  5  5
4  e  e  5  5
5  f  d  5  5 
 r2: 
    a  b  c  d
0  a  f  5  5
1  b  e  5  5
2  c  d  5  5 
 r3: 
    a  b  c  d
0  a  f  5  5
1  b  e  5  5
2  c  d  5  5


In [100]:
# 布尔运算符
df = pd.DataFrame(np.arange(9).reshape(3,3), columns = list('ABC'))
print(df)

df['Bool'] = df.eval('C>=5')
print(df)

r1 = df.query('not Bool')
r2 = (df.query('not Bool') == df[~df.Bool])
print('r1:', '\n', r1, '\n','r2:', '\n', r2)

   A  B  C
0  0  1  2
1  3  4  5
2  6  7  8
   A  B  C   Bool
0  0  1  2  False
1  3  4  5   True
2  6  7  8   True
r1: 
    A  B  C   Bool
0  0  1  2  False 
 r2: 
       A     B     C  Bool
0  True  True  True  True
